# Tethys Development

The purpose of this development notebook is to capture the relevant workflow that culminated in the table transformations generated from the Tethys API call, which are deployed and continually developed in the software package. This notebook will be out of date if the API changes, or significant changes are made to the codebase that deviate from the table decomposition outlined and arrived at by this notebook. It may be helpful, however, as a detailed look at what is possible to draw from the API by doing a git checkout to understand the development cycle of the table composition arrived at here. This may also be a helpful starting palce for additional feature development using the API, and a git checkout to this commit may give a good starting place for additional branches that take development into these features using this development cycle as a starting place.

### Setup

In [1]:
import requests
import pandas as pd
import re

tethys_api = 'https://tethys.pnnl.gov/api/primre_export'
tethys_response = requests.get(tethys_api)
tethys_response_json = tethys_response.json()
tethys_dataframe = pd.DataFrame(tethys_response_json)

### Dev

In [2]:
tethys_response_json[0]

{'URI': 'https://tethys.pnnl.gov/node/499',
 'type': ['Document', 'Document/Journal Article'],
 'landingPage': 'https://tethys.pnnl.gov/publications/marine-renewable-energy-mediterranean-sea-status-perspectives',
 'sourceURL': 'https://www.mdpi.com/1996-1073/10/10/1512/htm',
 'title': 'Marine Renewable Energy in the Mediterranean Sea: Status and Perspectives',
 'description': 'In this work, an extended overview of the marine renewable energy in the Mediterranean Sea is provided as regards current status, potential problems, challenges, and perspectives of development. An integrated and holistic approach is necessary for the economic viability and sustainability of marine renewable energy projects; this approach comprises three different frameworks, not always aligned, i.e., geotechnical/engineering, socio-economic, and environmental/ecological frameworks. In this context, the geomorphological, climatological, socio-economic, and environmental/ecological particularities of the Mediterra

In [15]:
tethys_dataframe[0:10]

,URI,type,landingPage,sourceURL,title,description,author,organization,originationDate,spatial,technologyType,tags,modifiedDate,signatureProject,attachment
0,https://tethys.pnnl.gov/node/499,"[Document, Document/Journal Article]",https://tethys.pnnl.gov/publications/marine-re...,https://www.mdpi.com/1996-1073/10/10/1512/htm,Marine Renewable Energy in the Mediterranean S...,"In this work, an extended overview of the mari...","[Soukissian, T., Denaxa, D., Karathanasi, F., ...","[Hellenic Centre for Marine Research (HCMR), N...",2017-09-29,[],[],"[Environment, Human Dimensions]",2024-01-22 09:24:45,[],[]
1,https://tethys.pnnl.gov/node/500,"[Document, Document/Report]",https://tethys.pnnl.gov/publications/port-fair...,http://bps.energy/projects,The Port Fairy Pilot Wave Energy Project Envir...,This Environmental Management Plan (EMP) detai...,[BioPower Systems],[BioPower Systems],2016-02-09,"{'coordinates': ['-38.398417000000', '142.1726...",[Wave],"[Environment, Environmental Impact Assessment]",2024-01-22 09:24:45,[],[https://tethys.pnnl.gov/sites/default/files/p...
2,https://tethys.pnnl.gov/node/501,"[Document, Document/Journal Article]",https://tethys.pnnl.gov/publications/baseline-...,https://www.sciencedirect.com/science/article/...,Baseline assessment of underwater noise in the...,The Ria Formosa is a sheltered large coastal l...,"[Soares, C., Pacheco, A., Zabel, F., González-...",[Marine Sensing and Acoustic Technologies (Mar...,2020-01-10,"{'coordinates': ['36.972554000000', '-7.870570...","[Current, Current/Tidal]","[Environment, Noise]",2024-01-22 09:24:45,[],[]
3,https://tethys.pnnl.gov/node/502,"[Document, Document/Journal Article]",https://tethys.pnnl.gov/publications/assessing...,https://www.sciencedirect.com/science/article/...,Assessing the consistency of in-stream tidal e...,"The provincial government of Nova Scotia, Cana...","[Carlson, J., Adams, M.]","[Queens University, Dalhousie University]",2020-03-01,"{'coordinates': ['45.166454000000', '-63.06410...","[Current, Current/Tidal]","[Environment, Human Dimensions, Legal & Policy]",2024-01-22 09:24:45,[],[]
4,https://tethys.pnnl.gov/node/503,"[Document, Document/Report]",https://tethys.pnnl.gov/publications/wave-ener...,https://publications.csiro.au/rpr/pub?pid=csir...,Wave Energy Deployments Physical Impact Guidel...,Australia’s wave energy resource has been asse...,"[McInnes, K., Hemer, M., O'Grady, J., Hoeke, R...",[Commonwealth Scientific and Industrial Resear...,2018-02-14,"{'coordinates': ['-32.203436000000', '115.6775...",[Wave],"[Environment, Physical Environment, Changes in...",2024-01-22 09:24:45,[],[https://tethys.pnnl.gov/sites/default/files/p...
5,https://tethys.pnnl.gov/node/504,"[Document, Document/Journal Article]",https://tethys.pnnl.gov/publications/large-sca...,https://kluwerlawonline.com/journalarticle/Eur...,Large-scale Water-related Innovative Renewable...,This article discusses two legal issues that r...,"[van Hees, S.]",[],2018-02-01,[],[],"[Environment, Birds, Human Dimensions, Legal &...",2024-01-22 09:24:45,[],[]
6,https://tethys.pnnl.gov/node/505,"[Document, Document/Report]",https://tethys.pnnl.gov/publications/environme...,,Environmental Monitoring Report EMEC AK-1000,This document provides a summary of the activi...,"[Rigg, D.]",[European Marine Energy Centre (EMEC)],2019-09-14,"{'coordinates': ['59.128445000000', '-2.803811...","[Current, Current/Tidal]","[Environment, Environmental Impact Assessment]",2024-01-22 09:24:45,[],[https://tethys.pnnl.gov/sites/default/files/p...
7,https://tethys.pnnl.gov/node/506,"[Document, Document/Report]",https://tethys.pnnl.gov/publications/behaviora...,https://scholarscompass.vcu.edu/etd/4866/,Behavioral responses of sub-adult Atlantic Stu...,This study sought to evaluate and quantify beh...,"[McIntyre, A., Janeski, T., Garman, G., Delogl...",[Virginia Commonwealth University],2016-11-06,[],[],"[Environment, Fish, Demersal Fish, EMF]",2024-01-22 09:24:45,[],[https://tethys.pnnl.gov/sites/default/f

### Clean

### Clean - Tests

In [4]:
def find_entry_id(entry_uri):
    '''
    This function takes in the url of a MHKDR entry, and returns the entry_id of that page. 
    The 'entry_id' is the integer at the end of the url, which is unique to each MHKDR entry.
    The regex used in this function relies on the fact that the only number in the url is the id.
    '''
    rule = re.compile(r'\d+')
    matches_rule = rule.findall(entry_uri)
    entry_id = int(matches_rule[0])

    return entry_id

In [5]:
def construct_authors_table(mhkdr_dataframe):
    '''
    This function creates a normalized table for the json element "author," connected to an "entry_id" that 
    may be called as a primary key to join this table to others. This disentangles the nested list structure
    present in the json to enable reporting e.g. associations among researchers, number of documents 
    attributed to each author.
    '''
    authors_of_entries = list(mhkdr_dataframe['author'])
    landing_page_uris = list(mhkdr_dataframe['URI'])
    
    entry_ids = list()  # This list will contain duplicate entry ids, as it represents the final column that will map to entry
    authors = list()    # This list will contain duplicate authors when an author contributes to multiple entries
    
    for i in range(0, len(mhkdr_dataframe)):
        # Construct "entry_id" - This will be a primary key for all future merge operations.
        entry_id = find_entry_id(landing_page_uris[i])
        
        # Construct "author" column
        num_authors = len(authors_of_entries[i])
        for j in range(0, num_authors):
            entry_ids.append(entry_id)
            authors.append(authors_of_entries[i][j])
    
    final_df = pd.DataFrame({'entry_id':entry_ids, 'author':authors})
    
    return final_df

In [10]:
def construct_organizations_table(mhkdr_dataframe):
    '''
    This function creates a normalized table for the json element "organization," connected to an "entry_id" that 
    may be called as a primary key to join this table to others. This disentangles the nested list structure
    present in the json to enable reporting e.g. associations among researchers, number of documents 
    attributed to each author.
    '''
    orgs_of_entries = list(mhkdr_dataframe['organization'])
    landing_page_uris = list(mhkdr_dataframe['URI'])
    
    entry_ids = list()  # This list will contain duplicate entry ids, as it represents the final column that will map to entry
    orgs = list()    # This list will contain duplicate authors when an author contributes to multiple entries
    
    for i in range(0, len(mhkdr_dataframe)):
        # Construct "entry_id" - This will be a primary key for all future merge operations.
        entry_id = find_entry_id(landing_page_uris[i])
        print(entry_id)
        print(orgs_of_entries[i])
        # Construct "organization" column
        org = orgs_of_entries[i][0]

        entry_ids.append(entry_id)
        orgs.append(org)
    
    final_df = pd.DataFrame({'entry_id':entry_ids, 'organization':orgs})
    
    return final_df

In [7]:
def construct_tags_table(mhkdr_dataframe):
    '''
    This function creates a normalized table for the json element "tags," connected to an "entry_id" that 
    may be called as a primary key to join this table to others. This disentangles the nested list structure
    present in the json to enable reporting e.g. associations among researchers, number of documents 
    attributed to each author.
    '''
    tags_of_entries = list(mhkdr_dataframe['tags'])
    landing_page_uris = list(mhkdr_dataframe['URI'])
    
    entry_ids = list()  # This list will contain duplicate entry ids, as it represents the final column that will map to entry
    tags = list()    # This list will contain duplicate authors when an author contributes to multiple entries
    
    for i in range(0, len(mhkdr_dataframe)):
        # Construct "entry_id" - This will be a primary key for all future merge operations.
        entry_id = find_entry_id(landing_page_uris[i])
        
        # Construct "tag" column
        num_tags = len(tags_of_entries[i])
        for j in range(0, num_tags):
            entry_ids.append(entry_id)
            tags.append(tags_of_entries[i][j])
    
    final_df = pd.DataFrame({'entry_id':entry_ids, 'tag':tags})
    
    return final_df

### Cleaned - Tests

In [8]:
construct_authors_table(tethys_dataframe)

,entry_id,author
0,499,"Soukissian, T."
1,499,"Denaxa, D."
2,499,"Karathanasi, F."
3,499,"Prospathopoulos, A."
4,499,"Sarantakos, K."
...,...,...
15817,2078886,"Williamson, B."
15818,2078886,"Couto, A."
15819,2078886,"Zampollo, A."
15820,2078886,"Davies, I."


In [11]:
construct_organizations_table(tethys_dataframe)

499
['Hellenic Centre for Marine Research (HCMR)', 'National Technical University of Athens']
500
['BioPower Systems']
501
['Marine Sensing and Acoustic Technologies (MarSensing)', 'University of Algarve']
502
['Queens University', 'Dalhousie University']
503
['Commonwealth Scientific and Industrial Research Organisation (CSIRO)']
504
[]


IndexError: list index out of range

In [ ]:
construct_tags_table(tethys_dataframe)